# Herramientas para la búsqueda de firmas de genes comunes con pares SL relevantes
```
Autor:  Martín Sende Pombo (email: martinsendepombo@outlook.com)
Se utilizó ChatGPT 3.5 como asistente de programación.
Creado: 11-06-2024
Última modificación: 13-08-2024
Propósito: Este cuaderno de Jupyter contiene herramientas destinadas a facilitar la identificación de las firmas de expresión génica, que superaron tanto el cribado con blitzGSEA como con la API de STRING.
```
Copyright (C) 2024  Martín Sende Pombo

    Este programa es software libre: puede redistribuirlo y/o modificarlo
    bajo los términos de la Licencia Pública General de GNU publicada por la
    Free Software Foundation, ya sea la versión 3 de la Licencia,
    o (a su elección) cualquier versión posterior.

    Este programa se distribuye con la esperanza de que sea útil,
    pero SIN NINGUNA GARANTÍA; ni siquiera la garantía implícita
    de COMERCIABILIDAD o IDONEIDAD PARA UN PROPÓSITO PARTICULAR.
    Consulte la Licencia Pública General GNU para más detalles.

    Debería haber recibido una copia de la Licencia Pública General
    GNU junto con este programa. Si no es así, consulte <https://www.gnu.org/licenses/>.

## Generación de listas de genes con relevancia estadísica y funcional o de interés

Este script recorre los archivos en la carpeta indicada (corresponden a firmas de expresión génica), elimina la terminación presente en sus nombres y guarda la lista resultante en un archivo .txt en la carpeta de destino, con cada nombre de archivo separado por una tabulación.

Ejemplos de nombres: blitzGSEA_HPV+p16_V1_CEGP_MSigDB, STRING_PT_V1_CEGP, DAISY_HPV-p16_V2_SoF

In [36]:
import os

def listar_archivos_y_guardar(origen, destino, nombre_archivo_salida, terminaciones_eliminar):
    # Ordenar las terminaciones de la más larga a la más corta para evitar coincidencias parciales incorrectas
    terminaciones_eliminar.sort(key=len, reverse=True)
    
    # Listar todos los archivos en la carpeta origen
    archivos = os.listdir(origen)
    
    # Crear una lista para almacenar los nombres de los archivos sin las terminaciones deseadas
    lista_nombres = []
    
    # Procesar cada archivo
    for archivo in archivos:
        # Verificar si el archivo tiene alguna de las terminaciones en la lista terminaciones_eliminar
        for terminacion in terminaciones_eliminar:
            if archivo.endswith(terminacion):
                # Eliminar la terminación y agregar el nombre a la lista
                nombre_sin_terminacion = archivo[: -len(terminacion)]
                lista_nombres.append(nombre_sin_terminacion)
                break
    
    # Crear la ruta completa para el archivo de salida
    ruta_archivo_salida = os.path.join(destino, nombre_archivo_salida)
    
    # Crear la carpeta de destino si no existe
    os.makedirs(destino, exist_ok=True)
    
    # Guardar la lista de nombres en un archivo .txt con nombres separados por tabulación
    with open(ruta_archivo_salida, 'w') as archivo_salida:
        archivo_salida.write("\t".join(lista_nombres))
    
    print(f"Archivo guardado en: {ruta_archivo_salida}")

# Ejemplo de uso
carpeta_origen = "Archivos_listar"
carpeta_destino = "Listas_genes_relevantes"
terminaciones_eliminar = ["_2_resultados_GSEA_significativos.xlsx", "_resultados_GSEA_significativos.xlsx"]
nombre_archivo_salida = input("Introduce el nombre del listado que se creará: ") + ".txt"

listar_archivos_y_guardar(carpeta_origen, carpeta_destino, nombre_archivo_salida, terminaciones_eliminar)

print("Proceso completado.")

Introduce el nombre del listado que se creará:  blitzGSEA_HPV+p16_V2_CEGP_MSigDB


Archivo guardado en: Listas_genes_relevantes\blitzGSEA_HPV+p16_V2_CEGP_MSigDB.txt
Proceso completado.


### Eliminación de archivos de la carpeta "Archivos_listar"

In [38]:
import os

# Carpeta que contiene lo archivos a borrar
carpeta = "Archivos_listar"

def eliminar_archivos(carpeta):
    # Listar todos los elementos en la carpeta
    for filename in os.listdir(carpeta):
        file_path = os.path.join(carpeta, filename)
        try:
            # Verificar si es un archivo y eliminarlo
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                #print(f'Archivo {file_path} eliminado')
            # Si es un directorio, lo ignoramos
        except Exception as e:
            print(f'No se pudo eliminar {file_path}. Razón: {e}')

# Eliminar archivos en la carpeta
eliminar_archivos(carpeta)
    
print("Proceso completado.")

Proceso completado.


## Obtención de genes comunes entre varias listas de genes

Este script lee todos los archivos .txt presentes en la carpeta Lista_genes_relevantes, encuentra las cadenas de caracteres comunes a todos esos archivos, y guarda los términos comunes en un nuevo archivo .txt denominado Interseccion_genes en la carpeta Listas_genes_definitivas. Si no se encuentra ningún gen común, se indicará en pantalla.

Ejemplos de nombres: DAISY_PT&HPV-_V2_SoF, STRING&blitzGSEA_HPV+_V1_CEGP_MSigDB, PT&HPV+(MSigDB)_V1_CEGP

In [44]:
import os

def leer_archivos_txt(carpeta):
    listas = []
    archivos = [f for f in os.listdir(carpeta) if f.endswith('.txt')]
    
    for archivo in archivos:
        ruta_archivo = os.path.join(carpeta, archivo)
        with open(ruta_archivo, 'r') as f:
            contenido = f.read()
            lista = contenido.split("\t")
            listas.append(set(lista))
    
    return listas

def encontrar_interseccion(listas):
    if not listas:
        return set()
    
    interseccion = listas[0]
    for lista in listas[1:]:
        interseccion &= lista
    
    return interseccion

def guardar_interseccion(interseccion, destino, nombre_archivo):
    ruta_archivo_salida = os.path.join(destino, nombre_archivo)
    
    # Crear la carpeta de destino si no existe
    os.makedirs(destino, exist_ok=True)
    
    if interseccion:
        with open(ruta_archivo_salida, 'w') as archivo_salida:
            archivo_salida.write("\t".join(interseccion))
        print(f"Archivo guardado en: {ruta_archivo_salida}")
    else:
        print("No se encontraron genes comunes en todas las listas.")

# Ejemplo de uso
carpeta_origen = "Listas_genes_relevantes"
carpeta_destino = "Listas_genes_comunes"
nombre_archivo_salida = input("Escriba un nombre para el archivo resultante de la intersección de los genes de las listas: ") + ".txt"

listas = leer_archivos_txt(carpeta_origen)
interseccion = encontrar_interseccion(listas)
guardar_interseccion(interseccion, carpeta_destino, nombre_archivo_salida)

print("Proceso completado.")

Escriba un nombre para el archivo resultante de la intersección de los genes de las listas:  PT&HPV+(MSigDB)_V2_CEGP


No se encontraron genes comunes en todas las listas.
Proceso completado.


## Vaciado de las carpetas de archivos
Tras copiar los archivos que considere de interés a otra ubicación, puede ejecutar este código para borrar todos los archivos presentes en las carpetas indicadas en la "Lista de las carpetas".

In [46]:
import os

# Lista de las carpetas
carpetas = [
    "Archivos_listar",
    "Listas_genes_relevantes",
    "Listas_genes_comunes"
]

def eliminar_archivos(carpeta):
    # Listar todos los elementos en la carpeta
    for filename in os.listdir(carpeta):
        file_path = os.path.join(carpeta, filename)
        try:
            # Verificar si es un archivo y eliminarlo
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
                print(f'Archivo {file_path} eliminado')
            # Si es un directorio, lo ignoramos
        except Exception as e:
            print(f'No se pudo eliminar {file_path}. Razón: {e}')

# Eliminar archivos en cada carpeta
for carpeta in carpetas:
    eliminar_archivos(carpeta)
    
print("Proceso completado.")

Archivo Listas_genes_relevantes\DAISY_PT&HPV+_V2_CEGP.txt eliminado
Archivo Listas_genes_relevantes\STRING&blitzGSEA_HPV+_V2_CEGP_MSigDB.txt eliminado
Archivo Listas_genes_relevantes\STRING&blitzGSEA_PT_V2_CEGP.txt eliminado
Proceso completado.
